Import the necessary libraries

In [4]:
import string 
import numpy as np
from PIL import Image
import os
from pickle import dump, load
import numpy as np

from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import add
from keras.models import Model, load_model
from keras.layers import Input, Dense, LSTM, Embedding, Dropout

from tqdm.notebook import tqdm as tqdm
tqdm().pandas()


0it [00:00, ?it/s]

# Cleaning The Data

We are using flicker8k dataset which contails 8091 images

In [5]:
"""The class File contains
load_file: Reads the given text file and returns the text
img_captions: Creates a dictionary which maps a file name to 5 captions
data_cleaning: Removes punctuation, removes words containing numbers and turns all alphabets into lowercase
get_vocabulary: Returns a set of all the words used in the captions
store_description: Map each caption to the text file name and store it in a text file
"""
class FileHandling:
    def __init__(self):
        pass
    def load_file(self, file):
        file = open(file,"r")
        text = file.read()
        file.close()
        return text
    def img_captions(self, file_contents):
        img_captions = dict()
        for file_name_raw in file_contents.split('\n'):
            file_name = file_name_raw.split('\t')[0].split('#')[0]
            if file_name not in img_captions.keys():
                img_captions[file_name] = []
            img_captions[file_name].append(file_name_raw.split('\t')[-1])
        return img_captions
    def data_cleaning(self, captions):
        #create a dictionary of ascii values of all punctuation mapped to none values
        img_edited_captions = {}
        punctuation_none_map = str.maketrans('','',string.punctuation)
        for img_name,all_captions in captions.items():
            if img_name not in img_edited_captions.values():
                img_edited_captions[img_name] = []
            for caption in all_captions:
                #remove punctuation
                caption = caption.translate(punctuation_none_map)
                all_words = caption.split()
                #convert to lower case
                all_words = [word.lower() for word in all_words]
                #remove hanging 's and a'
                all_words = [word for word in all_words if len(word) > 1]
                #remove words containing numbers
                all_words = [word for word in all_words if word.isalpha()]
                caption_edited = ' '.join(all_words)
                img_edited_captions[img_name].append(caption_edited)
        return img_edited_captions
    def get_vocabulary(self, captions):
        vocab = set()
        for img in captions.keys():
            for caption in captions[img]:
                vocab.update(caption.split())
        return vocab
            
    def store_description(self, all_captions, file_name):
        export_text = []
        for img,captions in all_captions.items():
            for caption in captions:
                export_text.append("{}\t{}".format(img,caption))
        file = open(file_name,"w")
        file.write('\n'.join(export_text))
        file.close()

In [6]:
File = FileHandling()
#File.load_file('Flickr8k_Text/Flickr8k.token.txt')
file_contents = File.load_file('Flickr8k_Text/Flickr8k.token.txt')
img_captions = File.img_captions(file_contents)
img_captions = File.data_cleaning(img_captions)
vocabulary = File.get_vocabulary(img_captions)
File.store_description(img_captions,'descriptions.txt')

# Extracting Features From Each Image

In [8]:
#extract_feature: Given a directory, this function extracts all the features using Xception model
def extract_features(directory):
    model = Xception(include_top = False, pooling = 'avg')
    features = {}
    for img in tqdm(os.listdir(directory)):
        file_name = directory+'/'+img
        image = Image.open(file_name)
        image = image.resize((299,299))
        image = np.expand_dims(image,axis=0)
        image = image / 127.5
        image = image - 1.0
        feature = model.predict(image)
        features[img] = feature
    return features

In [10]:
img_dataset = 'Flicker8k_Dataset'
features = extract_features(img_dataset)

  0%|          | 0/8091 [00:00<?, ?it/s]

1/1 [==============================] - 0s 69ms/step


1/1 [==============================] - 0s 89ms/step


1/1 [==============================] - 0s 81ms/step


1/1 [==============================] - 0s 75ms/step


...

1/1 [==============================] - 0s 107ms/step


1/1 [==============================] - 0s 89ms/step


1/1 [==============================] - 0s 76ms/step


1/1 [==============================] - 0s 76ms/step


1/1 [==============================] - 0s 91ms/step


In [11]:
dump(features, open("features.p","wb"))